In [ ]:
import pandas
print('pandas',pandas.__version__)
import numpy
print('numpy',numpy.__version__)
import glob
import matplotlib.pyplot as plt
import seaborn
from scipy import stats
import time

# load data from pickle file

In [ ]:
df = pandas.read_pickle("single_process_df_this_notebook_1552822411.6475844.pkl")

In [ ]:
print(df.shape)
df.head()

# drop column if entire column is constant value

In [ ]:
# https://stackoverflow.com/questions/39658574/how-to-drop-columns-which-have-same-values-in-all-rows-via-pandas-or-spark-dataf
series_of_unique_values_per_column = df.apply(pandas.Series.nunique)
cols_to_drop = series_of_unique_values_per_column[series_of_unique_values_per_column == 1].index
df = df.drop(cols_to_drop, axis=1)
print(df.shape)
df.head()

# scatter plot per column

In [ ]:
def scatter_plot_column(df,col_name):
    if not isinstance(df,pandas.core.frame.DataFrame):
        print('ERROR: expected input of type dataframe')
        return None
    print(col_name)
    x=df['time in unix seconds']
    y=df[col_name]
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

    print('r',r_value)
    print('p',p_value)
    print('std error:',std_err)

    _=plt.figure(figsize=(5,4))
    if (abs(1-r_value)<0.01):
        plt.scatter(x,y,label='original')
        plt.plot(x, intercept + slope*x, 'r', label='fitted line')
        linear=True
    else:
        x=df['datetime']
        plt.plot_date(x,y,label='original')
        plt.xticks(rotation=40)
        linear=False
    
    _=plt.ylabel(col_name,fontsize=14)
    _=plt.xlabel('time',fontsize=14)
    plt.show()
    return linear

In [ ]:
list_of_linear_columns=[]
for col_name in df.columns:
    if ('seconds' in col_name) or ('datetime' in col_name):
        print(col_name)
    else:
#        plt.scatter(x=df['time in unix seconds'],y=df[col_name])
        linear_bool = scatter_plot_column(df,col_name)
        if (linear_bool):
            list_of_linear_columns.append(col_name)

# drop columns that are linear

In [ ]:
list_of_linear_columns

In [ ]:
df = df.drop(list_of_linear_columns, axis=1)
print(df.shape)
df.head()

# find correlated columns and drop all but one of the correlated columns

https://stackoverflow.com/questions/39409866/correlation-heatmap


In [ ]:
df.columns

evaluate correlation for one pair

In [ ]:
df['CPU 1 times idle'].corr(df['process memory: rss'])

calculate the correlation matrix

In [ ]:
corr = df.corr()
corr.head()

In [ ]:
seaborn.set(rc={'figure.figsize':(10,8)})
# plot the heatmap 
# https://seaborn.pydata.org/generated/seaborn.heatmap.html
_=seaborn.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)#, annot=True)

# diagonal is 1, indicating perfect correlation
# black means anti-correlated

which columns are correlated?

In [ ]:
type(corr)

In [ ]:
# https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/

# Select upper triangle of correlation matrix
upper = corr.where(numpy.triu(numpy.ones(corr.shape), k=1).astype(numpy.bool))
#upper.head()

In [ ]:
for column in upper.columns:
    #print('column:',column)
    for row in upper.index:
        #print('row:',row)
        #print(upper[column].loc[row])
        if (upper[column].loc[row] > 0.999):
            print(column,'|',row,'|',upper[column].loc[row])
            _=scatter_plot_column(df,column)
            _=scatter_plot_column(df,row)

In [ ]:
seaborn.set(rc={'figure.figsize':(5,4)})
_=seaborn.regplot(x=df['CPU 1 times idle'], y=df['CPU 1 times softirq'])